In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using TensorFlow backend.


/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip


In [2]:
df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
rslt_df = df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)]
rslt_df2 = df[(df['toxic'] == 1) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)]
new1 = rslt_df[['id', 'comment_text', 'toxic']].iloc[:23891].copy() 
new2 = rslt_df2[['id', 'comment_text', 'toxic']].iloc[:946].copy()
new = pd.concat([new1, new2], ignore_index=True)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new["comment_text"], new['toxic'], test_size=0.33)

In [5]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(n_features=2**23)
X1 = vectorizer.fit_transform(X_train)
X_test1= vectorizer.transform(X_test)

In [7]:
print('Original dataset shape %s' % Counter(y_train))
sm = SVMSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X1, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


Original dataset shape Counter({0: 15994, 1: 646})
Resampled dataset shape Counter({0: 15994, 1: 15994})


LOGISTIC REGRESSION

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression(C=0.1, solver='sag')
scores = cross_val_score(clf2, X_res,y_res, cv=5,scoring='f1_weighted')
scores

array([0.80517004, 0.82717268, 0.85332295, 0.95063886, 0.95158277])

In [13]:
y_p1 = clf2.fit(X_res, y_res).predict(X_test1)
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_p1)
print('Accuracy: %f' % accuracy)

Accuracy: 0.906307


In [14]:
import numpy as np

z=1.96
interval = z * np.sqrt( (0.908137 * (1 - 0.908137)) / y_test.shape[0])
interval

0.0062528009129526945

SVC

In [15]:
from sklearn.svm import SVC
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf,X_res,y_res, cv=5)
scores

array([0.95514223, 0.97936855, 0.98546421, 0.98718149, 0.98827575])

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

y_p2 = clf.fit(X_res, y_res).predict(X_test1)

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_p2)
print('Accuracy: %f' % accuracy)

import numpy as np

z=1.96
interval = z * np.sqrt( (0.963279 * (1 - 0.963279)) / y_test.shape[0])
interval

Accuracy: 0.953154


0.004071569944781895

RANDOM FOREST

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# import numpy as np

# clf3 = RandomForestClassifier() #Initialize with whatever parameters you want to

# # 10-Fold Cross validation
# scores = cross_val_score(clf3,X_res,y_res, cv=5)
# scores

In [ ]:
# y_p3 = clf3.fit(X_res, y_res).predict(X_test1)
# accuracy = accuracy_score(y_test, y_p3)
# print('Accuracy: %f' % accuracy)

# z=1.96
# interval = z * np.sqrt( (0.9629 * (1 - 0.9629)) / y_test.shape[0])
# interval